In [1]:
from neo4j import GraphDatabase
import pandas as pd

In [2]:
# Neo4j connection info
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "testpassword"

# Connect
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [3]:
def run_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, parameters=params or {})
        return pd.DataFrame([r.data() for r in result])

## Total number of songs

In [4]:
query = "MATCH (s:Song) RETURN COUNT(s) AS total_songs"
run_query(query)

,total_songs
0,11530


## Top sampled songs

In [10]:
query = """
MATCH (:Song)-[r:SAMPLES]->(target:Song)
WITH target, COUNT(DISTINCT r) AS times_sampled
OPTIONAL MATCH (target)-[:HAS_ARTIST]->(artist:Artist)
WITH target.id AS song_id,
     target.title AS song_title,
     COLLECT(DISTINCT artist.name) AS artist_names,
     times_sampled
RETURN song_id, song_title, artist_names, times_sampled
ORDER BY times_sampled DESC
LIMIT 10
"""
run_query(query)

,song_id,song_title,artist_names,times_sampled
0,Lyn-Collins/Think-(About-It),Think (About It),[Lyn Collins],38
1,Beside/Change-the-Beat-(Female-Version),Change the Beat (Female Version),[Beside],31
2,"The-Winstons/Amen,-Brother","Amen, Brother",[The Winstons],27
3,Doug-E.-Fresh/La-Di-Da-Di,La Di Da Di,"[Slick Rick, Doug E. Fresh]",24
4,James-Brown/Funky-Drummer,Funky Drummer,[],24
5,Run-DMC/Here-We-Go-(Live-at-the-Funhouse),Here We Go (Live at the Funhouse),[],23
6,The-Honey-Drippers/Impeach-the-President,Impeach the President,[],21
7,Lil-Jon/Who-U-Wit%3F,Who U Wit?,"[The East Side Boyz, Lil Jon]",21
8,Mountain/Long-Red,Long Red,[Mountain],20
9,YG/I%27m-Good,I'm Good,[YG],17


## Genre distribution

In [14]:
query = """
MATCH (:Song)-[:BELONGS_TO_GENRE]->(g:Genre)
RETURN g.name AS genre, COUNT(*) AS count
ORDER BY count DESC
"""
run_query(query)

,genre,count
0,hip hop,2424
1,east coast hip hop,745
2,gangsta rap,544
3,electronic,539
4,rap,521
...,...,...
1400,rap/alternative and punk,1
1401,?,1
1402,jermaine,1
1403,rap hip-hop,1
